In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
from __future__ import print_function
from statsmodels.compat import lzip
import matplotlib.pyplot as plt
import numpy as np
import sklearn as sk
from sklearn.linear_model import LinearRegression

C:\Users\sanka\Anaconda2\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
from statsmodels.graphics.regressionplots import plot_fit
from statsmodels.graphics.regressionplots import plot_leverage_resid2
from statsmodels.graphics.regressionplots import plot_regress_exog
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import Ridge
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn.pipeline import make_pipeline

In [3]:
from sklearn.metrics import r2_score, mean_squared_error
linear_regression = linear_model.LinearRegression(normalize=False, fit_intercept=True)
def r2_est(X,y):
    return r2_score(y, linear_regression.fit(X,y).predict(X))

In [4]:
from sklearn.preprocessing import LabelEncoder, LabelBinarizer
from sklearn.decomposition import PCA
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.cross_validation import cross_val_score, KFold, StratifiedKFold
from sklearn.metrics import make_scorer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import RFECV
from sklearn.cross_validation import KFold
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import Lasso
from sklearn.grid_search import RandomizedSearchCV
from scipy.stats import expon

C:\Users\sanka\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\sanka\Anaconda2\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


## Data Loading and Preparation (time series)

In [5]:
cal = pd.read_csv('Datasources/inside_airbnb/calendar.csv')

In [6]:
cal['price'] = cal['price'].astype(str).map(lambda x: x.lstrip('$'))
cal['price'] = cal['price'].apply(pd.to_numeric, errors='coerce')
cal['price'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')

In [7]:
cal['date'] = pd.to_datetime(cal['date'])
cal['month'] = cal['date'].apply(lambda x: x.month)
cal['day'] = cal['date'].apply(lambda x: x.day)

In [8]:
cal['day_of_week'] = cal['date'].dt.weekday_name

In [9]:
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

In [10]:
cl = calendar()
holidays = cl.holidays(start=cal['date'].min(), end=cal['date'].max())

In [11]:
cal['holiday'] = cal['date'].isin(holidays)

In [12]:
c = cal.loc[cal.available!='f']
c = c[['listing_id','date','price','month','day_of_week','holiday']]

In [13]:
c_hol = c[c['holiday']==True]
c_wke = c[(c['holiday']==False)&((c['day_of_week']=='Sunday')|(c['day_of_week']=='Saturday'))]
c_wkd = c[(~c.isin(c_hol['date']))&(~c.isin(c_wke['date']))]

In [14]:
mean_price_hol = c_hol.groupby(by='listing_id')['price'].mean()
mean_price_wke = c_wke.groupby(by='listing_id')['price'].mean()
mean_price_wkd = c_wkd.groupby(by='listing_id')['price'].mean()

In [15]:
mean_price_hol = pd.DataFrame(mean_price_hol)
mean_price_wke = pd.DataFrame(mean_price_wke)
mean_price_wkd = pd.DataFrame(mean_price_wkd)

In [16]:
mean_price_hol = mean_price_hol.reset_index()
mean_price_wke = mean_price_wke.reset_index()
mean_price_wkd = mean_price_wkd.reset_index()

In [17]:
listings = pd.read_csv('Datasources/inside_airbnb/listings.csv')

In [18]:
master_hol = listings.merge(mean_price_hol, how='inner', left_on='id', right_on='listing_id')
master_wke = listings.merge(mean_price_wke, how='inner', left_on='id', right_on='listing_id')
master_wkd = listings.merge(mean_price_wkd, how='inner', left_on='id', right_on='listing_id')

## Feature Creation (time series)

In [19]:
master_hol['host_response_rate'] = master_hol['host_response_rate'].astype(str).map(lambda x: x.rstrip("%"))
master_hol['host_response_rate'] = master_hol['host_response_rate'].apply(pd.to_numeric, errors='coerce')
master_hol['host_response_rate'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
master_hol['host_acceptance_rate'] = master_hol['host_acceptance_rate'].astype(str).map(lambda x: x.rstrip("%"))
master_hol['host_acceptance_rate'] = master_hol['host_acceptance_rate'].apply(pd.to_numeric, errors='coerce')
master_hol['host_acceptance_rate'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
master_hol['extra_people'] = master_hol['extra_people'].astype(str).map(lambda x: x.lstrip('$'))
master_hol['extra_people'] = master_hol['extra_people'].apply(pd.to_numeric, errors='coerce')
master_hol['extra_people'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
master_hol['weekly_price'] = master_hol['weekly_price'].astype(str).map(lambda x: x.lstrip('$'))
master_hol['weekly_price'] = master_hol['weekly_price'].apply(pd.to_numeric, errors='coerce')
master_hol['weekly_price'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
master_hol['monthly_price'] = master_hol['monthly_price'].astype(str).map(lambda x: x.lstrip('$'))
master_hol['monthly_price'] = master_hol['monthly_price'].apply(pd.to_numeric, errors='coerce')
master_hol['monthly_price'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
master_hol['security_deposit'] = master_hol['security_deposit'].astype(str).map(lambda x: x.lstrip('$'))
master_hol['security_deposit'] = master_hol['security_deposit'].apply(pd.to_numeric, errors='coerce')
master_hol['security_deposit'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
master_hol['cleaning_fee'] = master_hol['cleaning_fee'].astype(str).map(lambda x: x.lstrip('$'))
master_hol['cleaning_fee'] = master_hol['cleaning_fee'].apply(pd.to_numeric, errors='coerce')
master_hol['cleaning_fee'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')

In [20]:
master_wke['host_response_rate'] = master_wke['host_response_rate'].astype(str).map(lambda x: x.rstrip("%"))
master_wke['host_response_rate'] = master_wke['host_response_rate'].apply(pd.to_numeric, errors='coerce')
master_wke['host_response_rate'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
master_wke['host_acceptance_rate'] = master_wke['host_acceptance_rate'].astype(str).map(lambda x: x.rstrip("%"))
master_wke['host_acceptance_rate'] = master_wke['host_acceptance_rate'].apply(pd.to_numeric, errors='coerce')
master_wke['host_acceptance_rate'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
master_wke['extra_people'] = master_wke['extra_people'].astype(str).map(lambda x: x.lstrip('$'))
master_wke['extra_people'] = master_wke['extra_people'].apply(pd.to_numeric, errors='coerce')
master_wke['extra_people'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
master_wke['weekly_price'] = master_wke['weekly_price'].astype(str).map(lambda x: x.lstrip('$'))
master_wke['weekly_price'] = master_wke['weekly_price'].apply(pd.to_numeric, errors='coerce')
master_wke['weekly_price'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
master_wke['monthly_price'] = master_wke['monthly_price'].astype(str).map(lambda x: x.lstrip('$'))
master_wke['monthly_price'] = master_wke['monthly_price'].apply(pd.to_numeric, errors='coerce')
master_wke['monthly_price'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
master_wke['security_deposit'] = master_wke['security_deposit'].astype(str).map(lambda x: x.lstrip('$'))
master_wke['security_deposit'] = master_wke['security_deposit'].apply(pd.to_numeric, errors='coerce')
master_wke['security_deposit'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
master_wke['cleaning_fee'] = master_wke['cleaning_fee'].astype(str).map(lambda x: x.lstrip('$'))
master_wke['cleaning_fee'] = master_wke['cleaning_fee'].apply(pd.to_numeric, errors='coerce')
master_wke['cleaning_fee'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')

In [21]:
master_wkd['host_response_rate'] = master_wkd['host_response_rate'].astype(str).map(lambda x: x.rstrip("%"))
master_wkd['host_response_rate'] = master_wkd['host_response_rate'].apply(pd.to_numeric, errors='coerce')
master_wkd['host_response_rate'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
master_wkd['host_acceptance_rate'] = master_wkd['host_acceptance_rate'].astype(str).map(lambda x: x.rstrip("%"))
master_wkd['host_acceptance_rate'] = master_wkd['host_acceptance_rate'].apply(pd.to_numeric, errors='coerce')
master_wkd['host_acceptance_rate'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
master_wkd['extra_people'] = master_wkd['extra_people'].astype(str).map(lambda x: x.lstrip('$'))
master_wkd['extra_people'] = master_wkd['extra_people'].apply(pd.to_numeric, errors='coerce')
master_wkd['extra_people'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
master_wkd['weekly_price'] = master_wkd['weekly_price'].astype(str).map(lambda x: x.lstrip('$'))
master_wkd['weekly_price'] = master_wkd['weekly_price'].apply(pd.to_numeric, errors='coerce')
master_wkd['weekly_price'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
master_wkd['monthly_price'] = master_wkd['monthly_price'].astype(str).map(lambda x: x.lstrip('$'))
master_wkd['monthly_price'] = master_wkd['monthly_price'].apply(pd.to_numeric, errors='coerce')
master_wkd['monthly_price'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
master_wkd['security_deposit'] = master_wkd['security_deposit'].astype(str).map(lambda x: x.lstrip('$'))
master_wkd['security_deposit'] = master_wkd['security_deposit'].apply(pd.to_numeric, errors='coerce')
master_wkd['security_deposit'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
master_wkd['cleaning_fee'] = master_wkd['cleaning_fee'].astype(str).map(lambda x: x.lstrip('$'))
master_wkd['cleaning_fee'] = master_wkd['cleaning_fee'].apply(pd.to_numeric, errors='coerce')
master_wkd['cleaning_fee'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')

### Amenities Feature Creation

In [22]:
def string_to_set(x):
    c = set()
    for w in x[1:-1].split(","):
        c.add(w)
        
    return c

In [23]:
master_hol['amenities_set'] = master_hol['amenities'].map(string_to_set)
master_wke['amenities_set'] = master_wke['amenities'].map(string_to_set)
master_wkd['amenities_set'] = master_wkd['amenities'].map(string_to_set)

In [24]:
def has_amenity(x, amen_):
    if amen_ in x:
        return 1
    pass

In [25]:
all_amenities = set()

for idx in listings['amenities'].map(string_to_set).index:
    all_amenities = all_amenities.union(listings['amenities'].map(string_to_set)[idx])

all_amenities

{'',
 '"24-Hour Check-in"',
 '"Air Conditioning"',
 '"Buzzer/Wireless Intercom"',
 '"Cable TV"',
 '"Carbon Monoxide Detector"',
 '"Elevator in Building"',
 '"Family/Kid Friendly"',
 '"Fire Extinguisher"',
 '"First Aid Kit"',
 '"Free Parking on Premises"',
 '"Hair Dryer"',
 '"Hot Tub"',
 '"Indoor Fireplace"',
 '"Laptop Friendly Workspace"',
 '"Lock on Bedroom Door"',
 '"Other pet(s)"',
 '"Pets Allowed"',
 '"Pets live on this property"',
 '"Safety Card"',
 '"Smoke Detector"',
 '"Smoking Allowed"',
 '"Suitable for Events"',
 '"Wheelchair Accessible"',
 '"Wireless Internet"',
 '"translation missing: en.hosting_amenity_49"',
 '"translation missing: en.hosting_amenity_50"',
 'Breakfast',
 'Cat(s)',
 'Dog(s)',
 'Doorman',
 'Dryer',
 'Essentials',
 'Gym',
 'Hangers',
 'Heating',
 'Internet',
 'Iron',
 'Kitchen',
 'Pool',
 'Shampoo',
 'TV',
 'Washer'}

In [26]:
for amen in all_amenities:
    master_hol['has' + amen] = 0
    master_wke['has' + amen] = 0
    master_wkd['has' + amen] = 0
    master_hol['has' + amen] = master_hol['amenities_set'].map(lambda x: has_amenity(x, amen))
    master_wke['has' + amen] = master_wke['amenities_set'].map(lambda x: has_amenity(x, amen))
    master_wkd['has' + amen] = master_wkd['amenities_set'].map(lambda x: has_amenity(x, amen))

In [27]:
has_amenties_list = []
for amen in all_amenities:
    has_amenties_list.append('has' + amen)

In [28]:
master_hol[has_amenties_list] = master_hol[has_amenties_list].fillna(0)
master_wke[has_amenties_list] = master_wke[has_amenties_list].fillna(0)
master_wkd[has_amenties_list] = master_wkd[has_amenties_list].fillna(0)

### Final Data Cleaning

In [29]:
X_hol = master_hol.select_dtypes(include=['float64', 'int64'])
X_wke = master_wke.select_dtypes(include=['float64', 'int64'])
X_wkd = master_wkd.select_dtypes(include=['float64', 'int64'])

In [30]:
target_hol = X_hol['price_y']
target_wke = X_wke['price_y']
target_wkd = X_wkd['price_y']

In [31]:
for w in X_hol.columns:
    v = np.isfinite(X_hol[w]).all()
    
    if v != True:
        X_hol = X_hol.drop(w, axis = 1)

In [32]:
for w in X_wke.columns:
    v = np.isfinite(X_wke[w]).all()
    
    if v != True:
        X_wke = X_wke.drop(w, axis = 1)

In [33]:
for w in X_wkd.columns:
    v = np.isfinite(X_wkd[w]).all()
    
    if v != True:
        X_wkd = X_wkd.drop(w, axis = 1)

In [34]:
X_hol = X_hol.join(target_hol, how = 'outer')
X_wke = X_wke.join(target_wke, how = 'outer')
X_wkd = X_wkd.join(target_wkd, how = 'outer')

In [35]:
X_hol = X_hol.dropna()
X_wke = X_wke.dropna()
X_wkd = X_wkd.dropna()

In [36]:
target_hol = X_hol['price_y']
target_wke = X_wke['price_y']
target_wkd = X_wkd['price_y']

In [37]:
X_hol = X_hol.drop('price_y', axis = 1)
X_wke = X_wke.drop('price_y', axis = 1)
X_wkd = X_wkd.drop('price_y', axis = 1)

#### Creating a whole target and predictors

In [38]:
target_whole = reduce(lambda x, y: x.combine_first(y), [target_hol, target_wke, target_wkd])
X_whole = reduce(lambda x, y: x.combine_first(y), [X_hol, X_wke, X_wkd])

## Modeling (time series)

### Model 1

#### Holidays

In [51]:
X_train, X_test, y_train, y_test = train_test_split(X_hol, target_hol, test_size=.30, random_state=1)
X_train_two, X_val, y_train_two, y_val = train_test_split(X_train, y_train, test_size=.30, random_state=1)

In [52]:
create_poly = PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)
poly = make_pipeline(create_poly, StandardScaler(), linear_regression)
model = poly.fit(X_train_two, y_train_two)
print('R^2 for validation is ' + str(r2_score(model.predict(X_val), y_val)))
print('Mean square error for validation is ' + str(mean_squared_error(model.predict(X_val), y_val)))
print('Mean absolute error for validation is ' + str(mean_absolute_error(model.predict(X_val), y_val)))
model_final = poly.fit(X_train, y_train)
print('R^2 for is ' + str(r2_score(model.predict(X_test), y_test)))
print('Mean square error is ' + str(mean_squared_error(model.predict(X_test), y_test)))
print('Mean absolute error is ' + str(mean_absolute_error(model.predict(X_test), y_test)))

R^2 for validation is -0.00283017560448
Mean square error for validation is 5.8175142112e+26
Mean absolute error for validation is 2.6023879429e+12
R^2 for is -0.00187731328572
Mean square error is 1.37285282279e+27
Mean absolute error is 3.89873370934e+12


#### Weekends

In [53]:
X_train, X_test, y_train, y_test = train_test_split(X_wke, target_wke, test_size=.30, random_state=1)
X_train_two, X_val, y_train_two, y_val = train_test_split(X_train, y_train, test_size=.30, random_state=1)

In [54]:
create_poly = PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)
poly = make_pipeline(create_poly, StandardScaler(), linear_regression)
model = poly.fit(X_train_two, y_train_two)
print('R^2 for validation is ' + str(r2_score(model.predict(X_val), y_val)))
print('Mean square error for validation is ' + str(mean_squared_error(model.predict(X_val), y_val)))
print('Mean absolute error for validation is ' + str(mean_absolute_error(model.predict(X_val), y_val)))
model_final = poly.fit(X_train, y_train)
print('R^2 for is ' + str(r2_score(model.predict(X_test), y_test)))
print('Mean square error is ' + str(mean_squared_error(model.predict(X_test), y_test)))
print('Mean absolute error is ' + str(mean_absolute_error(model.predict(X_test), y_test)))

R^2 for validation is -0.000772341051539
Mean square error for validation is 8.71006983678e+27
Mean absolute error for validation is 1.21553182891e+13
R^2 for is -0.00186413459864
Mean square error is 2.33413281969e+26
Mean absolute error is 1.34476990717e+12


#### Weekdays

In [55]:
X_train, X_test, y_train, y_test = train_test_split(X_wkd, target_wkd, test_size=.30, random_state=1)
X_train_two, X_val, y_train_two, y_val = train_test_split(X_train, y_train, test_size=.30, random_state=1)

In [56]:
create_poly = PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)
poly = make_pipeline(create_poly, StandardScaler(), linear_regression)
model = poly.fit(X_train_two, y_train_two)
print('R^2 for validation is ' + str(r2_score(model.predict(X_val), y_val)))
print('Mean square error for validation is ' + str(mean_squared_error(model.predict(X_val), y_val)))
print('Mean absolute error for validation is ' + str(mean_absolute_error(model.predict(X_val), y_val)))
model_final = poly.fit(X_train, y_train)
print('R^2 for is ' + str(r2_score(model.predict(X_test), y_test)))
print('Mean square error is ' + str(mean_squared_error(model.predict(X_test), y_test)))
print('Mean absolute error is ' + str(mean_absolute_error(model.predict(X_test), y_test)))

R^2 for validation is -1.48626451146e-08
Mean square error for validation is 4.59603897106e+26
Mean absolute error for validation is 2.16813679643e+12
R^2 for is -0.000117125099324
Mean square error is 1.36933072195e+26
Mean absolute error is 1.01603231134e+12


#### As a whole

In [57]:
X_train, X_test, y_train, y_test = train_test_split(X_whole, target_whole, test_size=.30, random_state=1)
X_train_two, X_val, y_train_two, y_val = train_test_split(X_train, y_train, test_size=.30, random_state=1)

In [58]:
create_poly = PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)
poly = make_pipeline(create_poly, StandardScaler(), linear_regression)
model = poly.fit(X_train_two, y_train_two)
print('R^2 for validation is ' + str(r2_score(model.predict(X_val), y_val)))
print('Mean square error for validation is ' + str(mean_squared_error(model.predict(X_val), y_val)))
print('Mean absolute error for validation is ' + str(mean_absolute_error(model.predict(X_val), y_val)))
model_final = poly.fit(X_train, y_train)
print('R^2 for is ' + str(r2_score(model.predict(X_test), y_test)))
print('Mean square error is ' + str(mean_squared_error(model.predict(X_test), y_test)))
print('Mean absolute error is ' + str(mean_absolute_error(model.predict(X_test), y_test)))

R^2 for validation is -0.00133875798284
Mean square error for validation is 2.4276924558e+28
Mean absolute error for validation is 8.11904812101e+12
R^2 for is -0.000293365790932
Mean square error is 1.72326505716e+27
Mean absolute error is 2.83132244835e+12
